In [4]:
import numpy as np

In [5]:
# angstroem, deg, kcal/mol, charge
# type, element, bond, angle, distance, energy, scale, charge
d_uff = np.dtype([
    ("type", np.unicode_, 8), 
    ("element", np.unicode_, 2), 
    ("bond", "d"),
    ("angle", "d"),
    ("distance", "d"),
    ("energy", "d"),
    ("scale", "d"),
    ("charge", "d"),
])
uff_data = np.loadtxt("uff.txt", dtype=d_uff)
#print(uff_data[0]["type"])

In [30]:
mass_dict = {}
with open("masses.txt") as f:
    for line in f.readlines():
        elem, mass = line.split()
        mass_dict[elem] = mass
#print(mass_dict)

In [33]:
with open("uff.xml", "w") as f:
    f.write("<ForceField>\n <AtomTypes>\n")
    for atom in uff_data:
        try:
            f.write(
                '  <Type name="{0}" class="{0}" element="{1}" mass="{2}" def="" desc=""/>\n'.format(
                    atom["type"], atom["element"], mass_dict[atom["element"]]
                )
            )
        except KeyError:
            pass # if we don't have the mass, skip it for now
    f.write(' </AtomTypes>\n <NonbondedForce coulomb14scale="0.833333" lj14scale="0.5">')
    for atom in uff_data:
        try:
            mass_dict[atom["element"]]
            f.write(
                '  <Atom type="{0}" charge="{1}" sigma="{2}" epsilon="{3}"/>\n'.format(
                    atom["type"], atom["charge"], atom["distance"], atom["energy"]
                )
            )
        except KeyError:
            pass
    f.write(' </NonbondedForce>\n</ForceField>')

In [2]:
!head -5 forcefield_template.xml

<ForceField>
 <AtomTypes>
  <Type name="opls_135" class="CT" element="C" mass="12.01100" def="[C;X4](H)(H)(H)C" desc="alkane CH3"/>
  <Type name="opls_136" class="CT" element="C" mass="12.01100" def="O" desc="alkane CH2"/>
  <Type name="opls_137" class="CT" element="C" mass="12.01100" def="O" desc="alkane CH"/>


In [12]:

1.54 - 0.757 *2


0.026000000000000023

In [15]:
with open("bonds.xml", "w") as f:
    f.write(" <HarmonicBondForce>\n")
    for i,atom1 in enumerate(uff_data):
        for atom2 in uff_data[i:]:
            # so in the rappe paper r_ij is supposed to be 
            # r_ij = r_i + r_j +r_BO + r_EN 
            # but to simplify things, I'm going to try just 
            # r_ij = r_i + r_j + 0.026
            # 0.026 is a correction so the c-c bond comes out right.
            length = atom1["bond"] + atom2["bond"] + 0.026 #angstroem
            
            # k_ij = 664.12 (Z*_i x Z*_j)/(r_ij^3) 
            energy = 664.12 * atom1["charge"] * atom2["charge"]/ length**3
            
            f.write(
                '  <Bond class1="{}" class2="{}" length="{:.3f}" k="{:.3f}"/>\n'.format(
                    atom1["type"], atom2["type"], length, energy
                )
            )

    f.write(' </HarmonicBondForce>')
